# Deliverable 2. Create a Customer Travel Destinations Map

----

1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Geoapify API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found".

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame. In the point for each city add:

    * The city name
    
    * The country code
    
    * The weather description
    
    * The point's size should be the maximum temperature for the city

15. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.

---

## Ensure that the dependencies and the Geoapify API key is imported correctly

In [10]:
# Dependencies and Setup
import sys
sys.path.append("./../../")
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")


In [11]:
# Import API key
from config import geoapify_key


## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`

In [19]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv('../WeatherPy_Database.csv')

# Display sample data
city_data_df.head(10)

,City,Country,Lat,Lng,Description,Humidity,MaxTemp,Cloudiness,WindSpeed
0,Anloga,GH,5.7947,0.8973,few clouds,79,78.31,22,4.07
1,Klyuchi,RU,52.2667,79.1667,overcast clouds,93,-9.29,100,11.21
2,Thinadhoo,MV,0.5333,72.9333,broken clouds,79,80.08,64,11.83
3,Punta Arenas,CL,-53.1500,-70.9167,clear sky,87,48.31,0,2.30
4,Mataura,NZ,-46.1927,168.8643,scattered clouds,46,63.43,38,7.16
5,Namibe,AO,-15.1961,12.1522,overcast clouds,77,70.02,94,6.31
6,Waipawa,NZ,-41.4122,175.5153,light rain,97,56.77,100,19.44
7,Bredasdorp,ZA,-34.5322,20.0403,broken clouds,76,66.31,60,3.47
8,East London,ZA,-33.0153,27.9116,fog,94,72.54,100,3.44
9,Leningradskiy,RU,69.3833,178.4167,broken clouds,99,-19.97,60,9.55


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation

In [20]:
# Prompt the user to enter minimum and maximum temperature criteria.

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90



## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`

In [21]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc method.
preferred_cities_df = city_data_df.loc[
    (city_data_df["MaxTemp"] <= max_temp) & (city_data_df["MaxTemp"] >= min_temp)
]


In [22]:
# Display sample data
preferred_cities_df.head(10)

,City,Country,Lat,Lng,Description,Humidity,MaxTemp,Cloudiness,WindSpeed
0,Anloga,GH,5.7947,0.8973,few clouds,79,78.31,22,4.07
2,Thinadhoo,MV,0.5333,72.9333,broken clouds,79,80.08,64,11.83
12,New Norfolk,AU,-42.7826,147.0587,scattered clouds,23,84.90,45,3.00
18,Avarua,CK,-21.2078,-159.7750,light rain,94,78.85,100,6.91
23,Kapaa,US,22.0752,-159.3190,scattered clouds,84,75.18,29,4.00
29,Pemangkat,ID,1.1667,108.9667,broken clouds,70,83.35,69,7.61
31,Rikitea,PF,-23.1203,-134.9692,light rain,77,78.75,75,15.82
38,Busselton,AU,-33.6500,115.3333,clear sky,43,78.08,0,15.21
40,Hobart,AU,-42.8794,147.3294,few clouds,54,84.27,20,13.80
41,Sabang,ID,5.8933,95.3214,overcast clouds,68,79.07,97,1.63


In [23]:
preferred_cities_df.count()

City           169
Country        168
Lat            169
Lng            169
Description    169
Humidity       169
MaxTemp        169
Cloudiness     169
WindSpeed      169
dtype: int64

## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows

In [24]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_travel_cities =preferred_cities_df.dropna(axis="index", how="any")
print(len(clean_travel_cities))
# Display sample data
clean_travel_cities.head(10)

168


,City,Country,Lat,Lng,Description,Humidity,MaxTemp,Cloudiness,WindSpeed
0,Anloga,GH,5.7947,0.8973,few clouds,79,78.31,22,4.07
2,Thinadhoo,MV,0.5333,72.9333,broken clouds,79,80.08,64,11.83
12,New Norfolk,AU,-42.7826,147.0587,scattered clouds,23,84.90,45,3.00
18,Avarua,CK,-21.2078,-159.7750,light rain,94,78.85,100,6.91
23,Kapaa,US,22.0752,-159.3190,scattered clouds,84,75.18,29,4.00
29,Pemangkat,ID,1.1667,108.9667,broken clouds,70,83.35,69,7.61
31,Rikitea,PF,-23.1203,-134.9692,light rain,77,78.75,75,15.82
38,Busselton,AU,-33.6500,115.3333,clear sky,43,78.08,0,15.21
40,Hobart,AU,-42.8794,147.3294,few clouds,54,84.27,20,13.80
41,Sabang,ID,5.8933,95.3214,overcast clouds,68,79.07,97,1.63


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng"

In [25]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Lat", "Lng", "Description", "Humidity", "MaxTemp", "Cloudiness", "WindSpeed"]].copy()


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame

In [26]:
# Create a new column "Hotel Name".
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Description,Humidity,MaxTemp,Cloudiness,WindSpeed,Hotel Name
0,Anloga,GH,5.7947,0.8973,few clouds,79,78.31,22,4.07,
2,Thinadhoo,MV,0.5333,72.9333,broken clouds,79,80.08,64,11.83,
12,New Norfolk,AU,-42.7826,147.0587,scattered clouds,23,84.90,45,3.00,
18,Avarua,CK,-21.2078,-159.7750,light rain,94,78.85,100,6.91,
23,Kapaa,US,22.0752,-159.3190,scattered clouds,84,75.18,29,4.00,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city

In [27]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

## Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found"

In [28]:
# Itirate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    #Get latitude and longitude from DF
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current's city lat and lon to the params dictionary
    params["filter"]= f"circle:{longitude},{latitude},{radius}"
    
    # Set up the base URL for the Geoapify Places API
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make a request and retrieve JSON data by using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert API response to JSON format
    name_address = name_address.json()

    
    # Get the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
    # If no hotel is found, set the hotel name as "No hotel found"
        hotel_df.loc[index, "Hotel Name"]= "No Hotel Found"
    
hotel_df.head(10)

,City,Country,Lat,Lng,Description,Humidity,MaxTemp,Cloudiness,WindSpeed,Hotel Name
0,Anloga,GH,5.7947,0.8973,few clouds,79,78.31,22,4.07,Hotel de White House
2,Thinadhoo,MV,0.5333,72.9333,broken clouds,79,80.08,64,11.83,No Hotel Found
12,New Norfolk,AU,-42.7826,147.0587,scattered clouds,23,84.90,45,3.00,Rosie's Inn
18,Avarua,CK,-21.2078,-159.7750,light rain,94,78.85,100,6.91,Natura Nikao Beach Bungalow
23,Kapaa,US,22.0752,-159.3190,scattered clouds,84,75.18,29,4.00,Sheraton Kauai Coconut Beach Resort
29,Pemangkat,ID,1.1667,108.9667,broken clouds,70,83.35,69,7.61,Hotel Duyung Permai 2
31,Rikitea,PF,-23.1203,-134.9692,light rain,77,78.75,75,15.82,Chez Bianca & Benoit
38,Busselton,AU,-33.6500,115.3333,clear sky,43,78.08,0,15.21,No Hotel Found
40,Hobart,AU,-42.8794,147.3294,few clouds,54,84.27,20,13.80,Hotel Grand Chancellor Hobart
41,Sabang,ID,5.8933,95.3214,overcast clouds,68,79.07,97,1.63,Sabang Hill


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`

In [29]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != "No Hotel Found"]

# Display sample data
clean_hotel_df.head(10)

,City,Country,Lat,Lng,Description,Humidity,MaxTemp,Cloudiness,WindSpeed,Hotel Name
0,Anloga,GH,5.7947,0.8973,few clouds,79,78.31,22,4.07,Hotel de White House
12,New Norfolk,AU,-42.7826,147.0587,scattered clouds,23,84.90,45,3.00,Rosie's Inn
18,Avarua,CK,-21.2078,-159.7750,light rain,94,78.85,100,6.91,Natura Nikao Beach Bungalow
23,Kapaa,US,22.0752,-159.3190,scattered clouds,84,75.18,29,4.00,Sheraton Kauai Coconut Beach Resort
29,Pemangkat,ID,1.1667,108.9667,broken clouds,70,83.35,69,7.61,Hotel Duyung Permai 2
31,Rikitea,PF,-23.1203,-134.9692,light rain,77,78.75,75,15.82,Chez Bianca & Benoit
40,Hobart,AU,-42.8794,147.3294,few clouds,54,84.27,20,13.80,Hotel Grand Chancellor Hobart
41,Sabang,ID,5.8933,95.3214,overcast clouds,68,79.07,97,1.63,Sabang Hill
44,Vaini,TO,-21.2000,-175.2000,broken clouds,79,87.96,75,6.91,Keleti International Resort
47,Mahebourg,MU,-20.4081,57.7000,light rain,78,81.25,75,3.44,Preskil Island Resort


## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the `Vacation_Search`` folder

In [30]:
# Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

## Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame.

In the point for each city add:

* The city name
* The country code
* The weather description
* The point's size should be the maximum temperature for the city

In [31]:
import geoviews as gv
from cartopy import crs

In [32]:
# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "MaxTemp",
    scale = 1,
    color = "City"
)

# Display the map plot
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,MaxTemp)